In [37]:
import models
import dataset
import tensorflow
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tensorflow.keras.optimizers import Adam

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
print("Hello world")

In [ ]:
class SaveModelCallback(tensorflow.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.model.save("saved_models/unet_segment")

In [ ]:
def get_data(feature_dir, label_dir):
    features = dataset.load_face_pictures(feature_dir, color_mode="rgb") / 255
    labels = dataset.load_face_pictures(label_dir, color_mode="grayscale") / 255

    features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2)

    return features_train, features_test, labels_train, labels_test

In [48]:
from cloud_ml.storage.api import Storage

# To retrieve application id and secret:
# 1. Go to link: https://oauth.yandex.ru/client/new
# 2. Choose 'Web services'
# 3. Paste into 'Callback URI': https://oauth.yandex.ru/verification_code
# 4. Set up permissions on yandex disk
disk = Storage.ya_disk(application_id='bf0cf4a358ba478787951f5095418e1d', application_secret='7b0d1705b6d846e78dd92fbefc1737ad')

# downloading contents of the remote file into the local one
disk.get('CelebA-HQ-img-256-256-masked.zip', 'CelebA-HQ-img-256-256-masked.zip')


In [49]:
import zipfile
from tqdm import tqdm

fname = './CelebA-HQ-img-256-256-masked.zip'
path = './medical/'

with zipfile.ZipFile(fname, 'r') as zf:
    for entry in tqdm(zf.infolist(), desc='Extracting '):
        try:
            zf.extract(entry, path)
        except zipfile.error as e:
            pass


Extracting : 100%|██████████| 29576/29576 [02:06<00:00, 233.11it/s]
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 29576
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 29576
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:882: UserWarning: The following variables cannot be serialized: disk, zf
  warnings.warn(message)


In [40]:
if __name__ == '__main__':
    input_layer = Input((256, 256, 3))
    model = models.build_model(input_layer, 16)

    model.compile(Adam(), loss='binary_crossentropy', metrics=["accuracy"])

    tc = tensorflow.keras.callbacks.TensorBoard(log_dir="tensorboard")
    smc = SaveModelCallback()

    features_train, features_test, labels_train, labels_test = get_data(
        "medical/CelebA-HQ-img-256-256-masked",
        "medical/CelebA-HQ-img-256-256-labels")

    model.fit(features_train, labels_train, batch_size=128, callbacks=[tc, smc], validation_split=0.1, epochs=300)

    predicted_labels = model.predict(features_test)

    print(f1_score(labels_test, predicted_labels, average=None))

/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 1
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'>
Walking trough too many objects
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}


NameError: name 'tensorflow' is not defined

In [9]:
%pip list


Package                  Version
------------------------ ------------------
absl-py                  0.12.0
addict                   2.4.0
antlr4-python3-runtime   4.8
appdirs                  1.4.4
argon2-cffi              20.1.0
astor                    0.8.1
asttokens                2.0.4
async-generator          1.10
atpublic                 2.1.3
attrs                    20.3.0
audioread                2.1.9
autograd                 1.3
aws-sam-translator       1.35.0
aws-xray-sdk             2.6.0
backcall                 0.2.0
bcrypt                   3.2.0
beautifulsoup4           4.9.3
bleach                   3.3.0
blis                     0.7.4
boto                     2.49.0
boto3                    1.12.31
botocore                 1.15.49
Bottleneck               1.3.2
cached-property          1.5.2
cachetools               4.2.1
catalogue                2.0.1
catboost                 0.24
certifi                  2020.12.5
cffi                     1.14.5
cfn-lint        

In [23]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import *
from tensorflow.keras.datasets import *
from sklearn.model_selection import train_test_split
import PIL.Image as Image
import dataset
import models
import tensorflow as tf

if __name__ == '__main__':

    dir = "medical/CelebA-HQ-img-256-256-masked"
    features = dataset.load_face_pictures(dir, img_num=100, color_mode='rgb') / 255

    model = load_model("saved_models/unet_segment.h5", compile=False)
    model.summary()
    predictions = model.predict(features)[:,:,:,0]

    for i in range(len(predictions)):
        predicted_img = Image.fromarray((predictions[i] * 255.0).astype("uint8"), "L")
        # real_label = Image.fromarray((l_test[i] * 255).astype("uint8"), "RGB")
        predicted_img.save("results/" + str(i) + ".png")
        # real_label.save("results/real_" + str(i) + ".png")

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 2320        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 16) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:882: UserWarning: The following variables cannot be serialized: model
  warnings.warn(message)


In [39]:
%pip install 'tensorflow==1.15.0' --force-reinstall

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl (412.3 MB)
  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)
  Using cached grpcio-1.36.1-cp37-cp37m-manylinux2014_x86_64.whl (4.1 MB)
  Using cached protobuf-3.15.6-cp37-cp37m-manylinux1_x86_64.whl (1.0 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
  Using cached numpy-1.20.2-cp37-cp37m-manylinux2010_x86_64.whl (15.3 MB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached gast-0.2.2-py3-none-any.whl
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached tensorflow_estimator-1.15.1-py2.py3-none-any.whl (503 kB)
  Using cached wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl
  Using cached termcolor-1.1.0-